<p style="float:right">
<img src="images/logos/cu.png" style="display:inline" />
<img src="images/logos/cires.png" style="display:inline" />
<img src="images/logos/nasa.png" style="display:inline" />
</p>

# Python, Jupyter & pandas: Module 4

## Using pandas for analysis

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt 

We can find a nice long in time, but very coarse dataset of northern hemisphere snowcover from David Robinson at Rutgers.

http://climate.rutgers.edu/snowcover/docs.php?target=datareq

_Robinson, David A., Estilow, Thomas W., and NOAA CDR Program (2012):NOAA Climate Date Record (CDR) of Northern Hemisphere (NH) Snow Cover Extent (SCE), Version 1. [indicate subset used]. NOAA National Climatic Data Center. doi:10.7289/V5N014G9 [access date]._

In [ ]:
import netCDF4

In [ ]:
snowcover_url = 'http://www.ncdc.noaa.gov/thredds/dodsC/cdr/snowcover/nhsce_v01r01_19661004_latest.nc'

open and connect the opendap endpoint 

In [ ]:
%%time 
ds = netCDF4.Dataset(snowcover_url)

In [ ]:
ds.ncattrs()

In [ ]:
ds.title

In [ ]:
ds.variables.keys()

In [ ]:
%%time
all_data = ds.variables['snow_cover_extent'][:, :, :]

In [ ]:
latitude = ds.variables['latitude']
longitude = ds.variables['longitude']


In [ ]:
land = ds.variables['land']

In [ ]:
plt.imshow(land[:], cmap='Accent')

In [ ]:
longitude[:].shape

In [ ]:
print(all_data.size, all_data.shape)

In [ ]:
plt.imshow(all_data[0,:,:], cmap='Accent', interpolation='nearest')

In [ ]:
from mpl_toolkits.basemap import Basemap

In [ ]:
plt.figure(figsize=(10, 10))
m = Basemap(projection='npstere', boundinglat=30, lon_0=-105)
m.drawcoastlines()
m.pcolor(longitude[:], latitude[:], land, latlon=True, cmap='Accent')
plt.draw()



In [ ]:
time = ds.variables['time']

In [ ]:
times = netCDF4.num2date(time[:], time.units)

In [ ]:
total_cells = a.sum(axis=0)

In [ ]:
with mpl.rc_context({'figure.figsize': (10,10)}):
    plt.imshow(total_cells, interpolation='nearest')

In [ ]:
import pandas as pd
import xarray as xr

In [ ]:
all_data.shape

In [ ]:
dset = xr.Dataset({'snowcover': (('time', 'row', 'col'), all_data)},
                  {'time': pd.DatetimeIndex(times)})

In [ ]:
dset.dims

In [ ]:
dset.time[0]

In [ ]:
dset.sel(time=['1966-10-10'])

In [ ]:
xr.DataArray(np.random.randn(2, 3))


In [ ]:
data = xr.DataArray(np.arange(6.).reshape(2, 3), [('x', ['a', 'b']), ('y', [-2, 0, 2])])


In [ ]:
data

In [ ]:
xr.DataArray(pd.Series(range(3), index=list('abc'), name='foo'))


In [ ]:
data.attrs

In [ ]:
data[:,[0,1,2]]                      # 

In [ ]:
data.loc[:,:]

In [ ]:
data.loc['b':'a':-1]

In [ ]:
data.isel(x=slice(0,2,1))

In [ ]:
data.sel(x='a')

In [ ]:
a = xr.DataArray(np.random.randn(3), [data.coords['y']])

In [ ]:
b = xr.DataArray(np.random.randn(4), dims='z')


In [ ]:
a

In [ ]:
b

In [ ]:
a + b

In [ ]:
data.T - data

In [ ]:
data[:-1] - data[:1]

In [ ]:
labels = xr.DataArray(['E', 'F', 'E'], [data.coords['y']], name='labels')


In [ ]:
labels

In [ ]:
data.groupby(labels).groups

In [ ]:
data

In [ ]:
data.groupby(labels).min('y').to_series()

In [ ]:
data

In [ ]:
data.to_series()

In [ ]:
dsex = data.to_dataset(name='foo')


In [ ]:
dsex

In [ ]:
dsex.to_netcdf('example.nc')

In [ ]:
ds2 = xr.open_dataset(snowcover_url)

In [ ]:
ds2

In [ ]:
ds2.coord_system